In [1]:
import pandas as pd
import numpy as np

In [2]:
all_divisions = {
    "Premier Main": 424,
    "2": 425,
    "3": 426,
    "4": 427,
    "5": 428,
    "6": 429,
    "7A": 430,
    "7B": 431,
    "8A": 432,
    "8B": 433,
    "9": 434,
    "10": 435,
    "11": 436,
    "12": 437,
    "13A": 438,
    "13B": 439,
    "14": 440,
    "15A": 441,
    "15B": 442,
    "Premier Masters": 443,
    "M2": 444,
    "M3": 445,
    "M4": 446,
    "Premier Ladies": 447,
    "L2": 448,
    "L3": 449,
    "L4": 450,
    }

In [3]:
summary_directory = r"C:\Users\bpali\PycharmProjects\SquashApp\2024-2025\summary_df"
schedules_directory = r"C:\Users\bpali\PycharmProjects\SquashApp\2024-2025\schedules_df"

In [4]:
awaiting_results_directory = r"C:\Users\bpali\PycharmProjects\SquashApp\2024-2025\awaiting_results"

In [5]:
# In the the schedules directory are week_x folders (eg week_4). First check the most recent folder (ie the highest number), then check the next highest, etc, 
# using the all_divisions dictionary to make sure we load every division's most recent summary file and avoid loading an older file for a division that has a more recent file.

schedules_dfs = []
for division in all_divisions.keys():
    for week in range(10, 0, -1):
        try:
            # Load csv file
            schedules_df = pd.read_csv(f"{schedules_directory}\\week_{week}\\{division}_schedules_df.csv")
            # Create division column
            schedules_df["Division"] = division
            # Append to list
            schedules_dfs.append(schedules_df)
            break
        except FileNotFoundError:
            continue

# Concatenate all the dataframes
schedules_df = pd.concat(schedules_dfs, ignore_index=True)

# Sort schedules_df by 'Division', then by 'Match Week', then by 'Date'
schedules_df = schedules_df.sort_values(by=['Division', 'Match Week', 'Date'])

# Save schedules_df to CSV
schedules_df.to_csv("updated_schedule_2024_2025.csv", index=False)

# Convert Date column to datetime
schedules_df['Date'] = pd.to_datetime(schedules_df['Date'], dayfirst=True)

In [6]:
# In the the summary directory are week_x folders (eg week_4). First check the most recent folder (ie the highest number), then check the next highest, etc, 
# using the all_divisions dictionary to make sure we load every division's most recent summary file and avoid loading an older file for a division that has a more recent file.

summary_dfs = []
for division in all_divisions.keys():
    for week in range(10, 0, -1):
        try:
            summary_df = pd.read_csv(f"{summary_directory}\\week_{week}\\{division}_summary_df.csv")
            summary_df["Division"] = division
            # Create Rank column based on Points column
            summary_df["Rank"] = summary_df["Points"].rank(ascending=False, method="min")
            # Make sure Rank column is an integer
            summary_df["Rank"] = summary_df["Rank"].astype(int)
            # Create column for number of teams in the division
            summary_df["Teams"] = len(summary_df)
            summary_dfs.append(summary_df)
            break
        except FileNotFoundError:
            continue

# Concatenate all the summary dataframes into one
summary_df = pd.concat(summary_dfs, ignore_index=True)

In [7]:
# In the the awaiting_results directory are week_x folders (eg week_4). First check the most recent folder (ie the highest number), then check the next highest, etc, 
# using the all_divisions dictionary to make sure we load every division's most recent summary file and avoid loading an older file for a division that has a more recent file.

awaiting_results_dfs = []
for division in all_divisions.keys():
    for week in range(10, 0, -1):
        try:
            awaiting_results_df = pd.read_csv(f"{awaiting_results_directory}\\week_{week}\\{division}_awaiting_results.csv")
            awaiting_results_df["Division"] = division
            awaiting_results_dfs.append(awaiting_results_df)
            break
        except FileNotFoundError:
            continue

# Concatenate all the awaiting_results dataframes into one
awaiting_results_df = pd.concat(awaiting_results_dfs)

### Create dataframe just for HKCC teams

In [8]:
hkcc = "Hong Kong Cricket Club"

# Filter rows where Team column contains "Hong Kong Cricket Club"
hkcc_summary_df = summary_df[(summary_df["Team"].str.contains(hkcc)) | 
                             (summary_df["Team"].str.contains("hkcc", case=False))].reset_index()

hkcc_summary_df[["Division", "Team", "Played", "Won", "Lost", "Points", "Rank", "Teams"]]

,Division,Team,Played,Won,Lost,Points,Rank,Teams
0,Premier Main,Hong Kong Cricket Club 1,5,4,1,13,2,6
1,2,Hong Kong Cricket Club 2,7,1,6,10,4,5
2,4,Hong Kong Cricket Club 3,9,3,6,20,6,8
3,6,Hong Kong Cricket Club 4,9,5,4,29,4,10
4,7A,Hong Kong Cricket Club 5,8,3,5,21,5,7
5,7B,Hong Kong Cricket Club 6,8,4,4,23,5,7
6,11,HKCC Tuesday Night Rockers,8,3,5,22,8,11
7,15A,Hong Kong Cricket Club 8,6,1,5,15,4,7
8,Premier Masters,Hong Kong Cricket Club M1B,7,4,3,15,3,5
9,Premier Masters,Hong Kong Cricket Club M1A,8,3,5,12,4,5


In [18]:
kcc = "Kowloon Cricket Club"

# Filter rows where Team column contains "Hong Kong Cricket Club"
kcc_summary_df = summary_df[(summary_df["Team"].str.contains(kcc)) | 
                             (summary_df["Team"].str.contains("kcc", case=False))].reset_index()

kcc_summary_df["Win %"] = kcc_summary_df["Won"] / kcc_summary_df["Played"] * 100

kcc_summary_df[["Division", "Team", "Played", "Won", "Lost", "Points", "Rank", "Teams", "Win %"]]

,Division,Team,Played,Won,Lost,Points,Rank,Teams,Win %
0,Premier Main,Kowloon Cricket Club 1A,5,5,0,17,1,6,100.000000
1,Premier Main,Kowloon Cricket Club 1B,5,0,5,3,6,6,0.000000
2,2,Kowloon Cricket Club 2,7,7,0,33,1,5,100.000000
3,3,Kowloon Cricket Club 3B,7,6,1,29,3,9,85.714286
4,3,Kowloon Cricket Club 3A,7,5,2,29,3,9,71.428571
5,4,Kowloon Cricket Club 4,9,7,2,39,2,8,77.777778
6,6,Kowloon Cricket Club 6B,9,9,0,45,1,10,100.000000
7,6,Kowloon Cricket Club 6A,9,5,4,30,3,10,55.555556
8,7B,Kowloon Cricket Club 7,8,6,2,30,3,7,75.000000
9,9,Kowloon Cricket Club 9,9,5,4,28,4,8,55.555556


### Filter schedules_df for HKCC teams

In [9]:
# Filter schedules_df for rows where Home Team column or Away Team column contains "Hong Kong Cricket Club" or "HKCC"
hkcc_schedules_df = schedules_df[(schedules_df["Home Team"].str.contains(hkcc)) |
                                    (schedules_df["Home Team"].str.contains("hkcc", case=False)) |
                                    (schedules_df["Away Team"].str.contains(hkcc)) |
                                    (schedules_df["Away Team"].str.contains("hkcc", case=False))]

### Create results_df from hkcc_schedules_df

In [10]:
def parse_result(result):
    """
    Function to parse the 'result' string
    """
    overall, rubbers = result.split('(')
    rubbers = rubbers.strip(')').split(',')
    return overall, rubbers

In [11]:
def count_games_won(row):
    """
    Function to count the number of games won by each team in a match,
    handling walkovers (WO) and conceded rubbers (CR) by referring to the 'Overall Score'.
    """
    home_games_won = 0
    away_games_won = 0

    # Calculate the games won from the rubbers, excluding 'CR' and 'WO'
    for rubber in row['Rubbers']:
        if rubber == 'CR' or rubber == 'WO':
            continue
        home, away = map(int, rubber.split('-'))
        home_games_won += home
        away_games_won += away

    # Now handle the 'WO' and 'CR' rubbers by referring to the 'Overall Score'
    if 'WO' in row['Rubbers'] or 'CR' in row['Rubbers']:
        home_overall_score, away_overall_score = map(int, row['Overall Score'].split('-'))
        
        # If the home team has a higher overall score, award the missing games to them
        # Otherwise, award the missing games to the away team
        for rubber in row['Rubbers']:
            if rubber == 'WO' or rubber == 'CR':
                if home_overall_score > away_overall_score:
                    home_games_won += 3
                else:
                    away_games_won += 3

    return home_games_won, away_games_won

In [12]:
# Exclude rows where 'Away Team' is '[BYE]' (indicative of a bye week)
results_df = hkcc_schedules_df[hkcc_schedules_df['Away Team'] != '[BYE]'].copy()

# Replace NaN values in 'Result' with an empty string before applying str.contains
results_df['Result'] = results_df['Result'].fillna('')

# Keep rows where 'Result' contains brackets (indicative of a played match)
results_df = results_df[results_df['Result'].str.contains(r'\(')]

# Apply the function to the 'Result' column
results_df[['Overall Score', 'Rubbers']] = results_df['Result'].apply(lambda x: pd.Series(parse_result(x)))

# Drop the original 'Result' column
results_df.drop(columns=['Result'], inplace=True)

# Replace 'CR' with NaN
results_df.replace('CR', np.nan, inplace=True)
results_df.replace('WO', np.nan, inplace=True)

# Count the number of Rubbers For and Against for each team

# Splitting the 'Overall Score' into two separate columns
results_df[['Home Score', 'Away Score']] = results_df['Overall Score'].str.split('-', expand=True).astype(int)

# Initialize dictionaries to keep track of won and conceded rubbers
rubbers_won = {}
rubbers_conceded = {}

# Create Games Won columns
results_df[['Home Games Won', 'Away Games Won']] = results_df.apply(count_games_won, axis=1, result_type='expand')

In [13]:
# Create Home Win column, using Home Games Won vs Away Games Won as tiebreaker
for i, row in results_df.iterrows():
    if row['Home Score'] > row['Away Score']:
        results_df.loc[i, 'Home Win'] = 1
    elif row['Home Score'] < row['Away Score']:
        results_df.loc[i, 'Home Win'] = 0
    else:
        if row["Home Games Won"] > row["Away Games Won"]:
            results_df.loc[i, 'Home Win'] = 1
        elif row["Home Games Won"] < row["Away Games Won"]:
            results_df.loc[i, 'Home Win'] = 0
        else:
            results_df.loc[i, 'Home Win'] = "Don't know"


C:\Users\bpali\AppData\Local\Temp\ipykernel_51432\3089069011.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Don't know' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  results_df.loc[i, 'Home Win'] = "Don't know"


In [14]:
results_df["Home Win"] = results_df["Home Win"].astype(int) # if this leads to error, we have an unknown 'tied' result to investigate
results_df.head()

ValueError: invalid literal for int() with base 10: "Don't know"

In [15]:
# Function to create Home Points and Away Points columns
def create_points_columns(df):

    df['Home Points'] = 0
    df['Away Points'] = 0

    for i, row in df.iterrows():
        if row['Home Score'] > row['Away Score']:
            df.at[i, 'Home Points'] = row["Home Score"] + 1
            df.at[i, 'Away Points'] = row["Away Score"]
        elif row['Home Score'] < row['Away Score']:
            df.at[i, 'Home Points'] = row["Home Score"]
            df.at[i, 'Away Points'] = row["Away Score"] + 1
        else:
            if row["Home Games Won"] > row["Away Games Won"]:
                df.at[i, 'Home Points'] = row["Home Score"] + 1
                df.at[i, 'Away Points'] = row["Away Score"]
            elif row["Home Games Won"] < row["Away Games Won"]:
                df.at[i, 'Home Points'] = row["Home Score"]
                df.at[i, 'Away Points'] = row["Away Score"] + 1
            else:
                print(f"Error: No winner found for {row['Home Team']} vs {row['Away Team']}")

    return df

# Apply the function to the dataframe
results_df = create_points_columns(results_df)

In [ ]:
results_df.head()

In [ ]:
# Convert Date column to datetime format and then filter for rows that are in November
results_df['Date'] = pd.to_datetime(results_df['Date'], dayfirst=True)
november_results_df = results_df[results_df['Date'].dt.month == 11]
november_results_df.head()

In [ ]:
# Create a dataframe similar to hkcc_summary_df but based on the results in november_results_df
# To do this, we aggregate the Home Team Home Points won and Away Team Away Points won and combine them
# We also aggregate the number of games played by each team

# Create a dictionary to store the points won by each team
points_won = {}
games_played = {}
games_won = {}

# Iterate over the rows in november_results_df
for i, row in november_results_df.iterrows():
    home_team = row['Home Team']
    away_team = row['Away Team']
    home_points = row['Home Points']
    away_points = row['Away Points']

    # If the team is not in the dictionary, add it
    if home_team not in points_won:
        points_won[home_team] = 0
    if away_team not in points_won:
        points_won[away_team] = 0

    # Add the points won to the dictionary
    points_won[home_team] += home_points
    points_won[away_team] += away_points

    # Add games played to the dictionary
    if home_team not in games_played:
        games_played[home_team] = 0
    if away_team not in games_played:
        games_played[away_team] = 0

    games_played[home_team] += 1
    games_played[away_team] += 1

    # Add games won to the dictionary
    if home_team not in games_won:
        games_won[home_team] = 0
    if away_team not in games_won:
        games_won[away_team] = 0
    
    if row["Home Win"] == 1:
        games_won[home_team] += 1
    else:
        games_won[away_team] += 1

# Create a dataframe from the dictionary
november_summary_df = pd.DataFrame(list(points_won.items()), columns=['Team', 'Points'])
november_summary_df['Played'] = november_summary_df['Team'].map(games_played)
november_summary_df['Won'] = november_summary_df['Team'].map(games_won)
november_summary_df['Lost'] = november_summary_df['Played'] - november_summary_df['Won']

# Drop Team rows that don't contain "Hong Kong Cricket Club" or "HKCC"
november_summary_df = november_summary_df[(november_summary_df["Team"].str.contains(hkcc)) |
                                          (november_summary_df["Team"].str.contains("hkcc", case=False))]

# Create Division column by using Team column of november_summary_df and hkcc_summary_df
november_summary_df["Division"] = november_summary_df["Team"].map(hkcc_summary_df.set_index("Team")["Division"])

november_summary_df[["Division", "Team", "Played", "Won", "Lost", "Points"]]

In [19]:
# If 'Home Team' contains 'Hong Kong Cricket Club' or 'HKCC', then summarize the Home Points won for each Home Team. Do the same for the Away Team.
# Then combine the two dataframes into one.

hkcc_home_points = results_df[results_df['Home Team'].str.contains(hkcc) 
                              | results_df['Home Team'].str.contains('hkcc', case=False)].groupby('Home Team')['Home Points'].sum()

hkcc_away_points = results_df[results_df['Away Team'].str.contains(hkcc) 
                                | results_df['Away Team'].str.contains('hkcc', case=False)].groupby('Away Team')['Away Points'].sum()

hkcc_points_df = pd.concat([hkcc_home_points, hkcc_away_points], axis=1).fillna(0)

In [ ]:
hkcc_points_df

In [ ]:
results_df[results_df["Home Score"] == 
            results_df["Away Score"]]

In [ ]:
round(hkcc_summary_df["Won"].sum() / hkcc_summary_df["Played"].sum(), 3)

### Show HKCC results that haven't been uploaded yet

In [23]:
# Filter rows where Home Team column or Away Team column contains "Hong Kong Cricket Club" or "hkcc" (case-insensitive)
hkcc_awaiting_results_df = awaiting_results_df[(awaiting_results_df["Home Team"].str.contains(hkcc, case=False)) | 
                                               (awaiting_results_df["Away Team"].str.contains(hkcc, case=False)) |
                                               (awaiting_results_df["Home Team"].str.contains("hkcc", case=False)) |
                                                (awaiting_results_df["Away Team"].str.contains("hkcc", case=False))]


In [ ]:
hkcc_awaiting_results_df.sort_values("Date")

In [ ]:
hkcc_schedules_df

In [26]:
# In the the schedules directory are week_x folders (eg week_4). First check the most recent folder (ie the highest number), then check the next highest, etc, 
# using the all_divisions dictionary to make sure we load every division's most recent summary file and avoid loading an older file for a division that has a more recent file.

old_schedules_dfs = []
for division in all_divisions.keys():
    for week in range(0, 5, 1):
        try:
            # Load csv file
            schedules_df = pd.read_csv(f"{schedules_directory}\\week_{week}\\{division}_schedules_df.csv")
            # Create division column
            schedules_df["Division"] = division
            # Append to list
            old_schedules_dfs.append(schedules_df)
            break
        except FileNotFoundError:
            continue

# Concatenate all the dataframes
old_schedules_df = pd.concat(old_schedules_dfs, ignore_index=True)

# Convert Date column to datetime
old_schedules_df['Date'] = pd.to_datetime(old_schedules_df['Date'], dayfirst=True)

# Convert all rows where 'Home Team' or 'Away Team' contains 'Hong Kong Cricket Club 7' to 'HKCC Tuesday Night Rockers'2024-2025'
old_schedules_df.loc[old_schedules_df['Home Team'].str.contains('Hong Kong Cricket Club 7'), 'Home Team'] = 'HKCC Tuesday Night Rockers'
old_schedules_df.loc[old_schedules_df['Away Team'].str.contains('Hong Kong Cricket Club 7'), 'Away Team'] = 'HKCC Tuesday Night Rockers'

In [27]:
# Create hkcc_old_schedules_df
hkcc_old_schedules_df = old_schedules_df[(old_schedules_df["Home Team"].str.contains(hkcc)) |
                                         (old_schedules_df["Away Team"].str.contains(hkcc)) |
                                         (old_schedules_df["Home Team"].str.contains("hkcc", case=False)) |
                                         (old_schedules_df["Away Team"].str.contains("hkcc", case=False))]

In [28]:
hkcc_schedules_df = hkcc_schedules_df.sort_values("Date", ascending=True)

In [29]:
hkcc_old_schedules_df = hkcc_old_schedules_df.sort_values("Date", ascending=True)

In [30]:
# Drop unnecessary columns
columns_to_drop = ["vs", "Time", "Result"]
hkcc_schedules_df.drop(columns=columns_to_drop, inplace=True)
hkcc_old_schedules_df.drop(columns=columns_to_drop, inplace=True)

# Drop rows where 'Away Team' is '[BYE]'
hkcc_schedules_df = hkcc_schedules_df[hkcc_schedules_df['Away Team'] != '[BYE]']
hkcc_old_schedules_df = hkcc_old_schedules_df[hkcc_old_schedules_df['Away Team'] != '[BYE]']

In [31]:
# Merge the two dataframes on 'Home Team' and 'Away Team', and "Date"
merged_df = pd.merge(hkcc_schedules_df, hkcc_old_schedules_df, on=["Home Team", "Away Team", "Date", "Division", "Venue"], suffixes=('', '_old'), how="outer")

In [32]:
# Sort the dataframe by 'Date'
merged_df = merged_df.sort_values("Date")

In [33]:
# Show rows where Match Week is NaN or Match Week_old is NaN
problem_df_new = merged_df[merged_df["Match Week"].isna()]
problem_df_old = merged_df[merged_df["Match Week_old"].isna()]



In [34]:
# Merge the two dataframes on 'Home Team' and 'Away Team', and "Venue"
merged_problem_df = pd.merge(problem_df_new, problem_df_old, on=["Home Team", "Away Team"], suffixes=('', '_new'), how="outer")

In [35]:
# Rename Date to "Old Date" and "Date_new" to "New Date"
merged_problem_df.rename(columns={"Date": "Old Date", "Date_new": "New Date"}, inplace=True)

In [36]:
merged_problem_df = merged_problem_df[["Home Team", "Away Team", "Old Date", "New Date", "Division"]]

In [ ]:
# Filter merged_problem_df for rows where 'Home Team' contains "hkcc" or "Hong Kong Cricket Club"
hkcc_problem_df = merged_problem_df[(merged_problem_df["Home Team"].str.contains(hkcc)) | 
                                    (merged_problem_df["Home Team"].str.contains("hkcc", case=False))]
hkcc_problem_df

In [ ]:
merged_problem_df